In [1]:
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import string
from trl import SFTTrainer, SFTConfig, DataCollatorForCompletionOnlyLM
from peft import LoraConfig
import torch

In [2]:
## モデル名

model_name = "cyberagent/calm2-7b-chat"

In [3]:
## トークナイザーのインポート

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast = True
)

In [4]:
## モデルのインポート
## リソースが足りないので、1/8量子化
## ファインチューニングする際のリソースを確保するために、多少精度を犠牲にする

quantization_config = BitsAndBytesConfig(load_in_4bit = True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = quantization_config,
    torch_dtype = torch.float32
)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model-00001-of-00002.safetensors:  18%|#8        | 1.85G/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.04G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [5]:
request_key = "USER:"
response_key = "ASSISTANT:"
chat_template = string.Template(
    "USER:${user}\nASSISTANT:${assistant}"
)

In [6]:
sample_text = chat_template.safe_substitute({
    "user": "人気タレント・タモリの本名は何でしょう？",
     "assistant": "" # AIに回答させるために空けておく
})

In [7]:
def generate(model, text):
    input_ids = tokenizer.encode(
        text,
        return_tensors = 'pt',
        add_special_tokens = True
    ).to(model.device)
    output_ids = model.generate(
        input_ids,
        max_new_tokens = 100,
        # do_sample = True,
        # temperature = 0.8,
    )
    print(
        tokenizer.decode(
            output_ids[0],
            skip_special_tokens = True
        )
    )

In [8]:
%%time
generate(model, sample_text)

USER:人気タレント・タモリの本名は何でしょう？
ASSISTANT: タモリの本名は森田一義です。
CPU times: user 676 ms, sys: 225 ms, total: 901 ms
Wall time: 918 ms


In [9]:
## 外部のデータセットをインポート

rs = "izumi-lab/llm-japanese-dataset"
datasets = load_dataset(
    rs,
    split = 'train'
)
datasets = datasets.train_test_split(test_size = 0.1)
train_datasets = datasets['train']
eval_datasets = datasets['test']

README.md:   0%|          | 0.00/3.21k [00:00<?, ?B/s]

data-cc-by-sa.jsonl:   0%|          | 0.00/2.38G [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9074340 [00:00<?, ? examples/s]

In [10]:
## 学習データセットの整形プロセス

def update_datasets(example):
    output_texts = []
    for instruction_, input_, output_ in zip(example['instruction'], example['input'], example['output']):
        text = chat_template.safe_substitute({
            "user": "{0}\n{1}".format(instruction_, input_),
            "assistant": "なんだいっ！！こんなこともわからないのかい！！仕方ないねっ！！！\n{0}".format(output_)
        })
        output_texts.append(text)
    return output_texts

In [11]:
## ??

collator = DataCollatorForCompletionOnlyLM(
    instruction_template = tokenizer.encode(request_key, add_special_tokens = False),
    response_template = tokenizer.encode(response_key, add_special_tokens = False),
    tokenizer = tokenizer
)

In [12]:
peft_config = LoraConfig(
    peft_type = 'LORA',
    task_type = 'CAUSAL_LM',
    r = 8,
    target_modules = 'all-linear',
    lora_alpha = 8,
    lora_dropout = 0.0,
)

In [13]:
## アダプタの付与
## 量子化したモデルをそのままファインチューニングできないため
## 全体チューニングではなく、出力換算するための行列を修飾する(Lora)
## 実際にtrainされるのは行列の部分だけ

model.add_adapter(peft_config)

In [14]:
%%time
generate(model, sample_text)

USER:人気タレント・タモリの本名は何でしょう？
ASSISTANT: タモリの本名は森田一義です。
CPU times: user 769 ms, sys: 12.7 ms, total: 781 ms
Wall time: 779 ms


In [15]:
%%time

## 訓練オブジェクトの定義

trainer = SFTTrainer(
    model,
    train_dataset = train_datasets.select(range(1000)),
    eval_dataset = eval_datasets.select(range(100)),
    args = SFTConfig(
        output_dir = "./tmp",
        evaluation_strategy = "epoch",
        per_device_train_batch_size = 4,
        per_device_eval_batch_size = 4,
        learning_rate = 1e-5,
        num_train_epochs = 3.0,
        lr_scheduler_type = "linear",
        warmup_ratio = 0.0,
        logging_strategy = "epoch",
        save_strategy = "epoch",
        report_to = "all",
        bf16 = True
    ),
    formatting_func = update_datasets,
    data_collator = collator,
    max_seq_length = 1024,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

CPU times: user 682 ms, sys: 47.1 ms, total: 729 ms
Wall time: 440 ms


↑警告文に従い後で修正

In [16]:
%%time

## 訓練実行

trainer.train()

Epoch,Training Loss,Validation Loss
1,2.061700,1.121226
2,1.057000,1.017121
3,1.008500,1.005729


/usr/local/lib/python3.11/dist-packages/transformers/integrations/peft.py:418: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/integrations/peft.py:418: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/integrations/peft.py:418: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/integrations/peft.py:418: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(


CPU times: user 5min 19s, sys: 2min 40s, total: 7min 59s
Wall time: 8min 2s


TrainOutput(global_step=750, training_loss=1.3757279052734375, metrics={'train_runtime': 482.2015, 'train_samples_per_second': 6.221, 'train_steps_per_second': 1.555, 'total_flos': 1.31387007762432e+16, 'train_loss': 1.3757279052734375, 'epoch': 3.0})

In [17]:
%%time
generate(model, sample_text)

USER:人気タレント・タモリの本名は何でしょう？
ASSISTANT:タモリの本名は「森田 一義」です。
CPU times: user 1.11 s, sys: 12.5 ms, total: 1.12 s
Wall time: 1.12 s
